In [14]:
import requests
import lxml.html
import re

#### - URL 목록 추출을 위한 scrape_list_page() 함수 정의
#### - scrape_list_page() 함수의 반환값은 list 처럼 반복 가능한 제너레이터로 구현

In [7]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [5]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [6]:
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

#### - 목차에 포함돼 있는 공백을 제거할 수 있는 normalize_space() 함수 정의
#### - List Comprehension 구문에 조건을 추가해서 빈 문자열을 제거

In [11]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    # scrape_list_page 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url) # Session을 사용해 상세페이지를 추출
        ebook = scrape_detail_page(response) # 상세페이지에서 상세정보를 추출
        print(ebook) # 상세 정보 출력
        break

In [15]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title':root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price':root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [16]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [17]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['\r\n\t\t<1편>\r\n', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '\r\n\t\t<2편>\r\n', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '\r\n\t\t<3편>\r\n', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '\r\n\t\t<4편>\r\n', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}
